In [1]:
import os
import fnmatch
from random import shuffle
import json
from config import config_io,original_datasets, pan21_format,processed
from pathlib import Path
import json
import pandas as pd

In [2]:
import sys
sys.executable

'/home/sukanya/anaconda3/envs/tf-gpu/bin/python'

In [3]:
# snippet to get percentage of same author and diff author
'''

training_data = pd.read_csv(config_io.get('pan_21_processed_train'))
print(training_data.shape)
training_data.head()
same_author = len(training_data[training_data['author_1']==training_data['author_2']])
diff_author = len(training_data[training_data['author_1']!=training_data['author_2']])
diff_author/(same_author + diff_author)'''

"\n\ntraining_data = pd.read_csv(config_io.get('pan_21_processed_train'))\nprint(training_data.shape)\ntraining_data.head()\nsame_author = len(training_data[training_data['author_1']==training_data['author_2']])\ndiff_author = len(training_data[training_data['author_1']!=training_data['author_2']])\ndiff_author/(same_author + diff_author)"

In [19]:
#data_path = config_io['pan_19_original_train']
data_path = original_datasets['valid_2021']
print(data_path)
os.listdir(data_path)
base = [file.split(".")[0] for file in os.listdir(data_path) if "txt" in file]
truth_files = ["truth-"+file+".json" for file in base]
problem_files = [file+".txt" for file in base] 
#file_list = [data_path+'/'+file for file in temp_file_list]

/home/sukanya/PhD/CLEF/2021/validation


In [20]:
truth_files

['truth-problem-1543.json',
 'truth-problem-631.json',
 'truth-problem-1839.json',
 'truth-problem-1554.json',
 'truth-problem-2399.json',
 'truth-problem-1068.json',
 'truth-problem-2182.json',
 'truth-problem-347.json',
 'truth-problem-378.json',
 'truth-problem-793.json',
 'truth-problem-2091.json',
 'truth-problem-2116.json',
 'truth-problem-287.json',
 'truth-problem-1973.json',
 'truth-problem-771.json',
 'truth-problem-1156.json',
 'truth-problem-719.json',
 'truth-problem-2125.json',
 'truth-problem-1270.json',
 'truth-problem-1257.json',
 'truth-problem-172.json',
 'truth-problem-2281.json',
 'truth-problem-2167.json',
 'truth-problem-899.json',
 'truth-problem-566.json',
 'truth-problem-377.json',
 'truth-problem-1325.json',
 'truth-problem-2172.json',
 'truth-problem-1534.json',
 'truth-problem-1117.json',
 'truth-problem-205.json',
 'truth-problem-367.json',
 'truth-problem-2247.json',
 'truth-problem-2092.json',
 'truth-problem-1494.json',
 'truth-problem-2304.json',
 'tru

In [6]:
def verify_file_exists(file_list,data_path):
    for file in file_list:
        if not Path(data_path+"/"+file).exists():
            print("Error! Could not find file: ",data_path+file)

In [7]:
verify_file_exists(truth_files,data_path)
verify_file_exists(problem_files,data_path)

In [21]:
def convertPAN_to_siamese_format(text,truth_data, problem_file):
    a1_list = []
    a2_list = []
    p1_list = []
    p2_list = []
    p_list = []
    paragraph_authors = truth_data['paragraph-authors']
    paragraphs = text.split("\n")
    non_zero_len_paragraphs = [para for para in paragraphs if len(para)>0] # problem-601 creates an empty paragraph causing the problem. so had to remove empty paragraphs
    if len(non_zero_len_paragraphs)!= len(paragraph_authors):
        print(f"Error! Number of paragraphs {len(non_zero_len_paragraphs)} does not equal paragraph authors {len(paragraph_authors)}! God help you now!")
        print(paragraph_authors)
        for para in non_zero_len_paragraphs:
            print(f"\n len: {len(para)} {para}")
            return None, None, None, None
    else:
        for i in range(0, len(paragraph_authors)):
            a1 = paragraph_authors[i]
            if i+1<len(paragraph_authors) and (i+1)< len(paragraphs):
                a2 = paragraph_authors[i+1]
                p1 = non_zero_len_paragraphs[i]
                p2 = non_zero_len_paragraphs[i+1]
                a1_list.append(a1)
                p1_list.append(p1)
                a2_list.append(a2)
                p2_list.append(p2)
                p_list.append(problem_file)
    return p_list, a1_list, a2_list, p1_list, p2_list

In [22]:
def read_text_json(problem_path, truth_path):
    with open(problem_path) as f:
        text = f.read()
    print("len:", len(text))
    with open(truth_path) as json_file:
        data = json.load(json_file)
    return text, data

In [23]:
a1_list = []
a2_list = []
p1_list = []
p2_list = []
p_list = []

for i in range(0,len(truth_files)): #['truth-problem-1432.json'] #
    truth_file = truth_files[i]
    problem_file =   problem_files[i] #'problem-1432.txt'
    print((truth_file, problem_file))
    text, data = read_text_json(data_path+"/"+problem_file, data_path+"/"+truth_file)
    p_list_temp, a1_list_temp, a2_list_temp, p1_list_temp, p2_list_temp = convertPAN_to_siamese_format(text,data, problem_file)
    if p_list_temp and a1_list_temp and a2_list_temp and p1_list_temp and p2_list_temp:
        p_list.extend(p_list_temp)
        a1_list.extend(a1_list_temp)
        a2_list.extend(a2_list_temp)
        p1_list.extend(p1_list_temp)
        p2_list.extend(p2_list_temp)
data = {"problem":p_list, "author_1":a1_list, "author_2":a2_list, "para1_text": p1_list, "para2_text":p2_list}
print(len(p_list), len( a1_list),  len( a2_list),  len( p1_list), len( p2_list))
df = pd.DataFrame(data)

('truth-problem-1543.json', 'problem-1543.txt')
len: 1046
('truth-problem-631.json', 'problem-631.txt')
len: 1887
('truth-problem-1839.json', 'problem-1839.txt')
len: 1142
('truth-problem-1554.json', 'problem-1554.txt')
len: 1626
('truth-problem-2399.json', 'problem-2399.txt')
len: 2957
('truth-problem-1068.json', 'problem-1068.txt')
len: 1132
('truth-problem-2182.json', 'problem-2182.txt')
len: 1150
('truth-problem-347.json', 'problem-347.txt')
len: 1792
('truth-problem-378.json', 'problem-378.txt')
len: 1344
('truth-problem-793.json', 'problem-793.txt')
len: 1915
('truth-problem-2091.json', 'problem-2091.txt')
len: 2684
('truth-problem-2116.json', 'problem-2116.txt')
len: 1591
('truth-problem-287.json', 'problem-287.txt')
len: 1190
('truth-problem-1973.json', 'problem-1973.txt')
len: 1013
('truth-problem-771.json', 'problem-771.txt')
len: 1273
('truth-problem-1156.json', 'problem-1156.txt')
len: 1591
('truth-problem-719.json', 'problem-719.txt')
len: 1260
('truth-problem-2125.json', 

len: 1121
('truth-problem-2248.json', 'problem-2248.txt')
len: 1243
('truth-problem-834.json', 'problem-834.txt')
len: 1618
('truth-problem-50.json', 'problem-50.txt')
len: 1498
('truth-problem-885.json', 'problem-885.txt')
len: 1985
('truth-problem-196.json', 'problem-196.txt')
len: 1522
('truth-problem-660.json', 'problem-660.txt')
len: 1090
('truth-problem-2062.json', 'problem-2062.txt')
len: 1998
('truth-problem-709.json', 'problem-709.txt')
len: 2702
('truth-problem-1149.json', 'problem-1149.txt')
len: 1190
('truth-problem-129.json', 'problem-129.txt')
len: 1256
('truth-problem-1716.json', 'problem-1716.txt')
len: 1808
('truth-problem-2232.json', 'problem-2232.txt')
len: 1075
('truth-problem-1844.json', 'problem-1844.txt')
len: 2094
('truth-problem-120.json', 'problem-120.txt')
len: 1390
('truth-problem-2017.json', 'problem-2017.txt')
len: 1899
('truth-problem-421.json', 'problem-421.txt')
len: 1604
('truth-problem-1031.json', 'problem-1031.txt')
len: 3107
('truth-problem-815.json

('truth-problem-1593.json', 'problem-1593.txt')
len: 1560
('truth-problem-1655.json', 'problem-1655.txt')
len: 2375
('truth-problem-1730.json', 'problem-1730.txt')
len: 2325
('truth-problem-1632.json', 'problem-1632.txt')
len: 1729
('truth-problem-1007.json', 'problem-1007.txt')
len: 2935
('truth-problem-284.json', 'problem-284.txt')
len: 1322
('truth-problem-74.json', 'problem-74.txt')
len: 1005
('truth-problem-1133.json', 'problem-1133.txt')
len: 1191
('truth-problem-507.json', 'problem-507.txt')
len: 1443
('truth-problem-1398.json', 'problem-1398.txt')
len: 3825
('truth-problem-1194.json', 'problem-1194.txt')
len: 2167
('truth-problem-872.json', 'problem-872.txt')
len: 1940
('truth-problem-3.json', 'problem-3.txt')
len: 2025
('truth-problem-1624.json', 'problem-1624.txt')
len: 1095
('truth-problem-704.json', 'problem-704.txt')
len: 1880
('truth-problem-972.json', 'problem-972.txt')
len: 1058
('truth-problem-197.json', 'problem-197.txt')
len: 1376
('truth-problem-1857.json', 'problem

len: 2165
('truth-problem-410.json', 'problem-410.txt')
len: 2161
('truth-problem-345.json', 'problem-345.txt')
len: 1843
('truth-problem-1014.json', 'problem-1014.txt')
len: 1493
('truth-problem-1918.json', 'problem-1918.txt')
len: 1259
('truth-problem-1930.json', 'problem-1930.txt')
len: 1220
('truth-problem-98.json', 'problem-98.txt')
len: 1006
('truth-problem-685.json', 'problem-685.txt')
len: 1013
('truth-problem-1562.json', 'problem-1562.txt')
len: 3244
('truth-problem-69.json', 'problem-69.txt')
len: 2179
('truth-problem-1779.json', 'problem-1779.txt')
len: 1954
('truth-problem-2168.json', 'problem-2168.txt')
len: 1072
('truth-problem-2395.json', 'problem-2395.txt')
len: 1256
('truth-problem-2095.json', 'problem-2095.txt')
len: 1276
('truth-problem-1985.json', 'problem-1985.txt')
len: 1024
('truth-problem-967.json', 'problem-967.txt')
len: 2777
('truth-problem-1751.json', 'problem-1751.txt')
len: 2739
('truth-problem-221.json', 'problem-221.txt')
len: 2180
('truth-problem-2113.j

('truth-problem-144.json', 'problem-144.txt')
len: 2093
('truth-problem-2076.json', 'problem-2076.txt')
len: 1101
('truth-problem-1517.json', 'problem-1517.txt')
len: 1737
('truth-problem-1551.json', 'problem-1551.txt')
len: 2225
('truth-problem-276.json', 'problem-276.txt')
len: 2468
('truth-problem-106.json', 'problem-106.txt')
len: 1027
('truth-problem-1047.json', 'problem-1047.txt')
len: 1107
('truth-problem-1626.json', 'problem-1626.txt')
len: 1222
('truth-problem-1876.json', 'problem-1876.txt')
len: 1189
('truth-problem-1956.json', 'problem-1956.txt')
len: 2412
('truth-problem-754.json', 'problem-754.txt')
len: 1678
('truth-problem-1675.json', 'problem-1675.txt')
len: 1134
('truth-problem-539.json', 'problem-539.txt')
len: 1165
('truth-problem-1377.json', 'problem-1377.txt')
len: 1236
('truth-problem-2020.json', 'problem-2020.txt')
len: 1302
('truth-problem-1831.json', 'problem-1831.txt')
len: 1969
('truth-problem-1610.json', 'problem-1610.txt')
len: 1208
('truth-problem-212.json

('truth-problem-1588.json', 'problem-1588.txt')
len: 1132
('truth-problem-2198.json', 'problem-2198.txt')
len: 2858
('truth-problem-1548.json', 'problem-1548.txt')
len: 1054
('truth-problem-1790.json', 'problem-1790.txt')
len: 2002
('truth-problem-779.json', 'problem-779.txt')
len: 2188
('truth-problem-1686.json', 'problem-1686.txt')
len: 1742
('truth-problem-614.json', 'problem-614.txt')
len: 1533
('truth-problem-30.json', 'problem-30.txt')
len: 1415
('truth-problem-1253.json', 'problem-1253.txt')
len: 1687
('truth-problem-668.json', 'problem-668.txt')
len: 1212
('truth-problem-2244.json', 'problem-2244.txt')
len: 1093
('truth-problem-513.json', 'problem-513.txt')
len: 1291
('truth-problem-1130.json', 'problem-1130.txt')
len: 3741
('truth-problem-1370.json', 'problem-1370.txt')
len: 1084
('truth-problem-1034.json', 'problem-1034.txt')
len: 1037
('truth-problem-1925.json', 'problem-1925.txt')
len: 2211
('truth-problem-1997.json', 'problem-1997.txt')
len: 2129
('truth-problem-755.json',

In [24]:
df.head()

,problem,author_1,author_2,para1_text,para2_text
0,problem-1543.txt,1,1,"Has anyone done this on Windows 8 or 8.1, usin...",We are trying to set up a web application in a...
1,problem-1543.txt,1,1,We are trying to set up a web application in a...,But this is not sufficient for us. When Chrom...
2,problem-631.txt,1,2,I kust realized that some other PC is also get...,"Because he leased IP address, most likely he j..."
3,problem-631.txt,2,2,"Because he leased IP address, most likely he j...","In general, I advice you against ever transmit..."
4,problem-631.txt,2,1,"In general, I advice you against ever transmit...","My mac is the one that is always on, Both the ..."


Approach
- first split text according to chnages position
- for each author excerpt check if same author cases can be generated..i.e split at paragraph change.

In [25]:
len(df)

14095

In [26]:
df.to_csv(processed['valid_2021'],index=False)